In [3]:
import pandas
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
# подготовка таблицы
data = pandas.read_csv('titanic.csv')
X_full = data[['Pclass', 'Fare', 'Age', 'Sex','Survived']]
X_non_Na = X_full.dropna() # удаление пустых
Y = X_non_Na.Survived
X = X_non_Na[['Pclass', 'Fare', 'Age', 'Sex']]
X_ = pandas.get_dummies(X,columns = ['Sex']) 
X = X_.drop('Sex_male',axis=1)


In [5]:
# простая проверочная таблица
my_sample = {'Pclass': 3, 'Fare': 7, 'Age': 32, 'Sex_female': 0}
may_sample_dataframe = pandas.DataFrame(data=[my_sample])

In [6]:
# предсказание простое
clf = RandomForestClassifier(random_state=241)
clf.fit(X, Y)
# расчет веса показателей
importances = clf.feature_importances_
print(X.columns)
print(importances)

Index(['Pclass', 'Fare', 'Age', 'Sex_female'], dtype='object')
[0.11289637 0.31074446 0.31715916 0.25920002]


In [36]:
# пример использования подбора различных вариантов
#clf = RandomForestClassifier(n_estimators=100,
#                               random_state= 50,
#                               max_features='sqrt',
#                               n_jobs=-1, verbose=1)

clf = RandomForestClassifier()

# здесь пишем какие параметры будет перебирать
#param_greed = {
#    "max_depth" : [2,5,10],
#    "max_features" : [2,5,10],
#    "splitter" : ['best', 'random']
#}

param_greed = {
    "max_depth" : [2,5,10],
    "criterion" : ['gini', 'entropy'],
    "min_samples_split" : [2,5,10],
    "min_samples_leaf" : [1,5,10],
}

In [37]:
# пишем каие параметры перебираем scoring = как оцениваем  ,
# cv = сколько тестовых выборок, n_jobs = -1 используем все процссоры
GS = GridSearchCV(clf,param_greed,scoring = 'roc_auc',cv = 6,n_jobs = -1)

In [38]:
GS.fit(X,Y)

GridSearchCV(cv=6, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [39]:
# лучший показатель
GS.best_score_

0.8639643273473038

In [40]:
# луяшие параметры
GS.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [41]:
# обращаемся
GS.best_estimator_.feature_importances_

array([0.16982371, 0.19899107, 0.16674373, 0.46444149])

In [42]:
# предсказываем
GS.best_estimator_.predict(may_sample_dataframe)

array([0])